In [3]:
import psycopg2

conn = psycopg2.connect(
    host="localhost", 
    database="solana",
    user="solana"
)

cur = conn.cursor()

# %%
cmd = """
select * from account
union 
select * from account_audit
order by slot, write_version asc
"""

import pandas as pd 
df = pd.read_sql_query(cmd, conn)
print("# of examples found:", len(df))

cur.close()
conn.close()

df.head()

# of examples found: 56


/Users/brennan/miniconda3/envs/10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,pubkey,owner,lamports,slot,executable,rent_epoch,data,write_version,updated_on,txn_signature
0,"[b'\xdf', b'\xac', b'+', b'1', b'\xe1', b'r', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",7906560,10,False,0,"[b'\xd8', b'\x92', b'k', b'^', b'h', b'K', b'\...",202,2022-10-19 13:18:59.690029,"[b'c', b'(', b'\xe0', b'\xbf', b'o', b'=', b'\..."
1,"[b'j', b'\xc8', b'\xb4', b'\x9c', b'\x9a', b'+...","[b'\x06', b'\xdd', b'\xf6', b'\xe1', b'\xd7', ...",2039280,11,False,0,"[b'>', b'q', b'm', b'j', b'k', b'p', b'\xe4', ...",211,2022-10-19 13:19:00.200585,"[b'\xf1', b'|', b'B', b'\x85', b'\xee', b""'"", ..."
2,"[b'n', b'\n', b'z', b'\xd5', b'\xfd', b'~', b'...","[b'\x06', b'\xdd', b'\xf6', b'\xe1', b'\xd7', ...",2039280,11,False,0,"[b'>', b'q', b'm', b'j', b'k', b'p', b'\xe4', ...",212,2022-10-19 13:19:00.200601,"[b'\xf1', b'|', b'B', b'\x85', b'\xee', b""'"", ..."
3,"[b'\xdf', b'\xac', b'+', b'1', b'\xe1', b'r', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",7906560,11,False,0,"[b'\xd8', b'\x92', b'k', b'^', b'h', b'K', b'\...",213,2022-10-19 13:19:00.200611,"[b'\xf1', b'|', b'B', b'\x85', b'\xee', b""'"", ..."
4,"[b'\xee', b'\x1b', b']', b'\xa4', b'\xbc', b'a...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",6514560,11,False,0,"[b'd', b'\xb1', b'\x08', b'k', b'\xa8', b'A', ...",214,2022-10-19 13:19:00.200623,"[b'\xf1', b'|', b'B', b'\x85', b'\xee', b""'"", ..."


In [4]:
from solana.publickey import PublicKey

datas = []
pks = []
for i, row in df.iterrows():
    pk = row.pubkey
    pk = PublicKey(b''.join([pk[i] for i in range(pk.nbytes)]))
    pks.append(str(pk))

    data = row.data
    data = b''.join([data[i] for i in range(data.nbytes)])
    datas.append(data)

df['_data'] = datas
df['pk'] = pks

In [5]:
from anchorpy import Program
from solana.publickey import PublicKey
from pathlib import Path 
from anchorpy import Idl 
import json 

file = Path('../driftpy/protocol-v2/target/idl/clearing_house.json')
with file.open() as f:
    idl_dict = json.load(f)
idl = Idl.from_json(idl_dict)
program = Program(idl, PublicKey(0), None)

import re 
with open(f'../driftpy/protocol-v2/programs/clearing_house/src/lib.rs', 'r') as f:
    data = f.read()
re_result = re.search('\[cfg\(not\(feature = \"mainnet-beta\"\)\)\]\ndeclare_id!\(\"(.*)\"\)', data)
program_id = PublicKey(re_result.group(1))
print('program pk:', program_id)

program pk: DUZwKJKAk2C9S88BYvQzck1M1i5hySQjxB4zW6tJ29Nw


In [6]:
import sys
sys.path.insert(0, '../driftpy/src/')

import json
from driftpy.addresses import * 

trial_outpath = 'tmp2'
# trial_outpath = 'lunaCrash'
with open(f"../backtest/{trial_outpath}/trial_no_oracle_guards/users.json", 'r') as f:
    users = json.load(f)

user_auths = [PublicKey(u) for u in users.values()]
users = [get_user_account_public_key(program_id, auth) for auth in user_auths]
user_stats = [
    get_user_stats_account_public_key(program_id, pk)
    for pk in user_auths
]

# vaults 
i = 0
spot_vault_public_key = get_spot_market_vault_public_key(program_id, i)
insurance_vault_public_key = get_insurance_fund_public_key(program_id, i)

account_pks = {
    'State': get_state_public_key(program_id),
    'PerpMarket': get_perp_market_public_key(program_id, 0), # only single market rn
    'User': users,
    'UserStats': user_stats,
}
len(users)

3

In [13]:
save_path = Path('data')/trial_outpath
save_path.mkdir(exist_ok=True, parents=True)
print(save_path.absolute())

sys.path.insert(0, '../')
from backtest.helpers import serialize_perp_market_2
from spl.token._layouts import ACCOUNT_LAYOUT

def extract(pk, type):
    type_rows = df[df['pk'] == str(pk)]
    type_history = []
    rows = []
    for i, r in type_rows.iterrows():
        data = r['_data']
        type_obj = program.account[type]._coder.accounts.parse(data).data
        type_history.append(type_obj)
        rows.append(r)
    return type_history, rows

# vaults 
i = 0
spot_vaults = []
type_rows = df[df['pk'] == str(spot_vault_public_key)]
for _, r in type_rows.iterrows():
    data = r['_data']
    obj = ACCOUNT_LAYOUT.parse(data)
    obj = dict(obj) 
    obj.pop('_io')

    # todo things 
    # add time information to dict
    obj['slot'] = r['slot']
    obj['write_version'] = r['write_version']
    obj['updated_on'] = r['updated_on']

    spot_vaults.append(obj)
spot_vault_df = pd.DataFrame(spot_vaults)
spot_vault_df.to_csv(save_path/f"spot_vault_{i}.csv", index=False)

i = 0
insurance_vaults = []
type_rows = df[df['pk'] == str(insurance_vault_public_key)]
for _, r in type_rows.iterrows():
    data = r['_data']
    obj = ACCOUNT_LAYOUT.parse(data)
    obj = dict(obj) 
    obj.pop('_io')

    # todo things 
    # add time information to dict
    obj['slot'] = r['slot']
    obj['write_version'] = r['write_version']
    obj['updated_on'] = r['updated_on']

    insurance_vaults.append(obj)
insurance_vault_df = pd.DataFrame(insurance_vaults)
insurance_vault_df.to_csv(save_path/f"insurance_vault_{i}.csv", index=False)

# accounts 
i = 0
perp_market_history, rows = extract(account_pks['PerpMarket'], 'PerpMarket')
perp_market_df = []
for perp_market, r in zip(perp_market_history, rows):
    perp_df = serialize_perp_market_2(perp_market)

    # add time information to dict
    perp_df['slot'] = r['slot']
    perp_df['write_version'] = r['write_version']
    perp_df['updated_on'] = r['updated_on']

    perp_market_df.append(perp_df)

perp_market_df = pd.concat(perp_market_df)
perp_market_df.to_csv(save_path/f"perp_market_{i}.csv", index=False)

i = 0
state_history, rows = extract(account_pks['State'], 'State')
state_df = []
for state, r in zip(state_history, rows):
    d = state.__dict__
    # todo do things
            
    # add time information to dict
    d['slot'] = r['slot']
    d['write_version'] = r['write_version']
    d['updated_on'] = r['updated_on']

    state_df.append(d)
state_df = pd.DataFrame(state_df)
state_df.to_csv(save_path/f"state_{i}.csv", index=False)

for i, (user, user_stat) in enumerate(zip(users, user_stats)):
    user_stats_history, stats_rows = extract(user_stat, "UserStats")
    user_history, user_rows = extract(user, "User")

    user_df = []
    for user, r in zip(user_history, user_rows):
        d = user.__dict__

        # add time information to dict
        d['slot'] = r['slot']
        d['write_version'] = r['write_version']
        d['updated_on'] = r['updated_on']

        # todo do things
        user_df.append(d)
    user_df = pd.DataFrame(user_df)
    user_df.to_csv(save_path/f"user_{i}.csv", index=False)

    user_stats_df = []
    for user_s, r in zip(user_stats_history, stats_rows):
        d = user_s.__dict__

        # add time information to dict
        d['slot'] = r['slot']
        d['write_version'] = r['write_version']
        d['updated_on'] = r['updated_on']

        # todo do things
        user_stats_df.append(d)
    user_stats_df = pd.DataFrame(user_stats_df)
    user_stats_df.to_csv(save_path/f"user_stats_{i}.csv", index=False)

/Users/brennan/Documents/drift/py-drift/drift-sim/solana-accountsdb-plugin-postgres/data/tmp2
